### via europe code 

In [1]:
asins = ['B0038ZIY9I',
'B0041L70Q6',
'B00509CI38',
'B005F8U502',
'B00B4RT5HW',
'B00WDZ9HVI',
'B01CYMJEAS',
'B01JRXTOIE',
'B01N9LYY90',
'B06ZXZWB8F',
'B072XMZ7J9',
'B073LYWDJH',
'B074MYSX1Y',
'B074P1FX4K',
'B075DR9ZPT',
'B076DS6BT4']


In [2]:
import requests
import json
import pandas as pd
import openpyxl

## rainforest 

In [10]:
# Read the Excel file into a pandas DataFrame
import pandas as pd
df = pd.read_excel('../documents/Taric Code_Test file.xlsx')
df =df[~df['ASIN'].isna()]

In [11]:
asins_df = list(df.ASIN)
asins_df

['B09RDXKH4V',
 'B0BMG63KL4',
 'B09C823KH1',
 'B08GYMPKSQ',
 'B08L5QRS6N',
 'B092HYB96D',
 'B07FPKPQT5',
 'B09WLV6BZ5',
 'B09LV5PQ9Q',
 'B09QKTLJY9']

In [21]:
asin = 'B076DS6BT4'
asin = 'B0041L70Q6'
asin = 'B0BMG63KL4'# test on one asin 
rainforest_api_key = '2BE30AF36DC7483AA7FE7E9CA64F46FD'
amazon_domain = 'amazon.de'

In [19]:
def scrap_asin (asin ,rainforest_api_key , amazon_domain  ):
    params = {
              'api_key': rainforest_api_key,
              'amazon_domain': amazon_domain,
              'asin': asin,
              'type': 'product'}
    api_result = requests.get('https://api.rainforestapi.com/request', params)
    api_json = api_result.json()
    return api_json 

In [14]:
def extract_features(api_json) : 
    specs = api_json['product']['specifications']
    title = api_json['product']['title']
    columns = [x['name'] for x in specs]
    values = [x['value'] for x in specs]
    df_spec = pd.DataFrame([values], columns=columns)
    df_spec['title'] = title
    return df_spec

In [22]:

api_json_ = scrap_asin (asin ,rainforest_api_key , amazon_domain  )

In [23]:
api_json_['request_info']['success']

True

In [154]:
for key in api_json_['product'].keys() : 
    print(key)

title
search_alias
keywords
keywords_list
asin
link
brand
sell_on_amazon
categories
categories_flat
description
sub_title
rating
rating_breakdown
ratings_total
main_image
images
images_count
images_flat
is_bundle
attributes
top_reviews
buybox_winner
specifications
specifications_flat
color
weight
first_available
dimensions
model_number


In [25]:
api_json_['product']['title']

'Colexy Frischhaltedose für Aufschnitt Fresh System, Lebensmittelkonservierung Lebensmittelbehälter, Frischhaltedose zur Aufbewahrung von Wurs, Optimale Aufbewahrung im Kühlschrank, 22x15x5cm'

In [26]:
api_json_['product']

{'title': 'Colexy Frischhaltedose für Aufschnitt Fresh System, Lebensmittelkonservierung Lebensmittelbehälter, Frischhaltedose zur Aufbewahrung von Wurs, Optimale Aufbewahrung im Kühlschrank, 22x15x5cm',
 'search_alias': {'title': 'Baumarkt', 'value': 'diy'},
 'keywords': 'Colexy,Frischhaltedose,für,Aufschnitt,Fresh,System,,Lebensmittelkonservierung,Lebensmittelbehälter,,Frischhaltedose,zur,Aufbewahrung,von,Wurs,,Optimale,Aufbewahrung,im,Kühlschrank,,22x15x5cm',
 'keywords_list': ['Colexy',
  'Frischhaltedose',
  'Aufschnitt',
  'Fresh',
  'System',
  'Lebensmittelkonservierung',
  'Lebensmittelbehälter',
  'Frischhaltedose',
  'Aufbewahrung',
  'Wurs',
  'Optimale',
  'Aufbewahrung',
  'Kühlschrank',
  '22x15x5cm'],
 'asin': 'B0BMG63KL4',
 'link': 'https://www.amazon.de/Colexy-Frischhaltedose-Lebensmittelkonservierung-Lebensmittelbeh%C3%A4lter-Aufbewahrung/dp/B0BMG63KL4',
 'brand': 'Colexy',
 'sell_on_amazon': True,
 'categories': [{'name': 'Baumarkt'},
  {'name': 'Küche, Haushalt & W

In [31]:
asin_example_col = extract_features(api_json_).columns
asin_example_col

Index(['Hersteller', 'Produktabmessungen', 'Modellnummer', 'Farbe',
       'Spezielle Eigenschaften', 'Lieferumfang', 'Batterien notwendig',
       'Artikelgewicht', 'ASIN', 'Amazon Bestseller-Rang',
       'Im Angebot von Amazon.de seit', 'Marke', 'Farbe', 'Produktabmessungen',
       'Anzahl der Sets', 'Besonderes Merkmal', 'Ist Spülmaschinenfest',
       'Anzahl der Einheiten', 'title'],
      dtype='object')

In [160]:
extract_features(api_json_)

,Package Dimensions,Item model number,Department,Date First Available,Manufacturer,ASIN,title
0,3.39 x 2.91 x 0.24 inches; 0.71 Ounces,2540-00150,unisex-adult,"April 9, 2014",Mammut,B0038ZIY9I,Mammut Dual Adjust Side Squeeze Buckle


### automate on all asin 

In [58]:
%%time
df_all_asin = pd.DataFrame()
#for asin in asins_df : 
for count, asin in enumerate(asins_df):
    print(asin)
   
    api_json_product = scrap_asin (asin ,rainforest_api_key , amazon_domain )
    if api_json_product['request_info']['success'] == True:
        df_product = extract_features(api_json_product)
        df_product['index_'] = count
        #try :
        df_product.set_index('index_')
        #except :
         #   continue
        df_product = df_product.loc[~df_product.index.duplicated(keep='first')]
        df_all_asin = df_all_asin.loc[~df_all_asin.index.duplicated(keep='first')]
        #df_product = df_product.reindex(sorted(df_product.columns), axis=1)
        #df_all_asin = df_all_asin.append(df_product, ignore_index=True)
        df_all_asin = pd.concat([df_all_asin, df_product])
        
        #df_all_asin = df_all_asin.append(df_product)
    else  : 
        print('cannot scrapp  asin number : ', asin )
        print( 'the message is : '  ,  api_json_product['request_info']['message'])

B09RDXKH4V
B0BMG63KL4


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [62]:
%%time
df_all_asin = pd.DataFrame()
#for asin in asins_df : 
for count, asin in enumerate(asins_df):
    print(asin)
   
    api_json_product = scrap_asin (asin ,rainforest_api_key , amazon_domain )
    if api_json_product['request_info']['success'] == True:
        df_product = extract_features(api_json_product)
        df_product.to_csv('../outputs/prduct_'+str(asin)+'.csv', index =False )
        
        
    else  : 
        print('cannot scrapp  asin number : ', asin )
        print( 'the message is : '  ,  api_json_product['request_info']['message'])

B09RDXKH4V
B0BMG63KL4
B09C823KH1
B08GYMPKSQ
B08L5QRS6N
B092HYB96D
B07FPKPQT5
B09WLV6BZ5
B09LV5PQ9Q
B09QKTLJY9
CPU times: user 176 ms, sys: 28.7 ms, total: 205 ms
Wall time: 1min 54s


In [68]:
# Import libraries
import glob
import pandas as pd

# Get CSV files list from a folder
path = '../outputs/'
csv_files = glob.glob(path + "/*.csv")
csv_files

['../outputs/prduct_B09C823KH1.csv',
 '../outputs/prduct_B09LV5PQ9Q.csv',
 '../outputs/prduct_B08L5QRS6N.csv',
 '../outputs/prduct_B09WLV6BZ5.csv',
 '../outputs/prduct_B08GYMPKSQ.csv',
 '../outputs/prduct_B07FPKPQT5.csv',
 '../outputs/prduct_B0BMG63KL4.csv',
 '../outputs/prduct_B092HYB96D.csv',
 '../outputs/prduct_B09RDXKH4V.csv',
 '../outputs/prduct_B09QKTLJY9.csv']

In [69]:
# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file) for file in csv_files)
df_list

<generator object <genexpr> at 0x7fe9685ba4a0>

In [70]:
# Concatenate all DataFrames
big_df   = pd.concat(df_list, ignore_index=True)

In [71]:
big_df

,Hersteller,Artikelnummer,Produktabmessungen,Modellnummer,Farbe,Anzahl der Packungen,Batterien inbegriffen,Batterien notwendig,Artikelgewicht,ASIN,...,Controller-Typ,Anzahl der Lichtquellen,Artikelmaße L x B x H,Artikelmenge pro Paket,Batterien,Verbindungstechnologien,Anschlusstyp,Kompatible Telefonmodelle,Ampere,USB-Anschlüsse insgesamt
0,‎Pyatofyy,‎347320,‎6.5 x 6 x 6 cm; 573 Gramm,‎347320,‎Silber,‎1,‎Nein,‎Nein,‎573 g,B09C823KH1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,‎KYSUN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,‎100 g,B09LV5PQ9Q,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,‎10,NaN,NaN,NaN,B08L5QRS6N,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,‎35 x 35 x 9.7 cm; 8.6 Kilogramm,NaN,‎A8+,NaN,NaN,NaN,‎8.6 kg,B09WLV6BZ5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,‎20 x 15 x 15 cm; 300 Gramm,NaN,NaN,NaN,NaN,NaN,NaN,B08GYMPKSQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,‎Amoyeric,‎DDCK1102,‎40 x 22.5 x 5.5 cm; 500 Gramm,‎DDCK1102,‎Schwarz,NaN,‎Nein,‎Nein,‎500 g,B07FPKPQT5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,‎Colexy,NaN,‎21 x 13 x 5.5 cm; 170 Gramm,‎ghyh67u,‎Grüne,NaN,NaN,‎Nein,‎170 g,B0BMG63KL4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,‎yiyicas,NaN,NaN,NaN,NaN,NaN,‎Nein,‎Nein,‎284 g,B092HYB96D,...,Tastensteuerung,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,‎Pyatofyy,NaN,‎29 x 8.5 x 0.5 cm; 290 Gramm,NaN,‎Wie Gezeigt,NaN,‎Nein,NaN,290 Gramm,‎B09RDXKH4V,...,NaN,NaN,29 x 8.5 x 0.5 cm,1.0,NaN,NaN,NaN,NaN,NaN,NaN
9,FitFasting,NaN,19 x 19 x 10.01 cm; 580 Gramm,CS-PB-983S-Black,Black819,NaN,NaN,NaN,NaN,B09QKTLJY9,...,NaN,NaN,NaN,NaN,4 Lithium-Polymer Batterien erforderlich.,USB,Lightning,"Smartphones, Samsung iPhone",3 Ampere,2.0
